<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/LAB03_Pr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Importación de librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.function_base import linspace
from math import floor
from math import ceil
from numpy.ma.core import size

from sklearn import tree
from sklearn.datasets import load_iris
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

#Descarga y lectura de datos

In [51]:
# Lectura de la base de datos
filename = 'lluvias.csv'
data = pd.read_csv(filename, sep = ',')

###Revisión y limpieza de datos
Observamos que tenemos un dataset con un total de 25551 registros y solo existen datos NaN en 3 de ellos, es decir, está bastante completo.

In [52]:
# Revisión de los datos
print(data.shape)

data.info()
data.head(10)

(25551, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25551 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25551 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25551 non-null  int64  
 3   TMIN    25551 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 998.2+ KB


,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
5,1948-01-06,0.44,48,39,True
6,1948-01-07,0.41,50,40,True
7,1948-01-08,0.04,48,35,True
8,1948-01-09,0.12,50,31,True
9,1948-01-10,0.74,43,34,True


Decidimos eliminar los Nan ya que son solo 3 y no será significativo para el estudio que vamos a realizar. Y quitamos los duplicados, que como podemos observar, en este caso no los hay.

In [53]:
# Elimino los Nan.
data = data.dropna()

# Quitamos los duplicados
data = data.drop_duplicates()

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25548 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25548 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25548 non-null  int64  
 3   TMIN    25548 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ MB


Nos damos cuenta de que el valor máximo de la temperatura es muy alto, por lo que entendemos que los registros están en grados Fahrenheit. 

Realizamos la conversión de todas las temperaturas a grados Celsius, con la fórmula de conversión.

In [54]:
temp_max = data["TMAX"].max()

print('La temperatura máxima encontrada es', temp_max,'entendemos que estamos en Fº y queremos trabajar en Cº.')

La temperatura máxima encontrada es 103 entendemos que estamos en Fº y queremos trabajar en Cº.


In [55]:
data["TMIN"] = (data.TMIN - 32)*(5/9)

data["TMAX"] = (data.TMAX - 32)*(5/9)

temp_max = data["TMAX"].max().round(2)

temp_min = data["TMIN"].min().round(2)

print('La temperatura máxima en grados Celsius es', temp_max,'Cº.')

La temperatura máxima en grados Celsius es 39.44 Cº.


Imprimimos los primeros 40 registros para comprobar que se hayan modificado correctamente.

In [56]:
data.head(40)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,10.555556,5.555556,True
1,1948-01-02,0.59,7.222222,2.222222,True
2,1948-01-03,0.42,7.222222,1.666667,True
3,1948-01-04,0.31,7.222222,1.111111,True
4,1948-01-05,0.17,7.222222,0.000000,True
5,1948-01-06,0.44,8.888889,3.888889,True
6,1948-01-07,0.41,10.000000,4.444444,True
7,1948-01-08,0.04,8.888889,1.666667,True
8,1948-01-09,0.12,10.000000,-0.555556,True
9,1948-01-10,0.74,6.111111,1.111111,True


Categorizamos las columnas 'TMIN', 'TMAX' y 'PRCP' tal y como se pide en el enunciado del ejercicio.

'TMIN' tomará las etiquetas 'Baja', 'Media' y 'Alta', en función de los rangos de temperatura < 10, 10 >= temeperatura <= 20 y temperatura > 20.

In [57]:
conditions = [
          (data['TMIN'] < 10),
          ((data['TMIN'] >= 10) & (data['TMIN'] <= 20)),
          (data['TMIN'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMIN'] = np.select(conditions, results)

'TMAX' se categoriza igual que 'TMIN'.

In [58]:
conditions = [
          (data['TMAX'] < 10),
          ((data['TMAX'] >= 10) & (data['TMAX'] <= 20)),
          (data['TMAX'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMAX'] = np.select(conditions, results)

In [59]:
# Vemos cuantos registros de cada etiqueta tiene TMIN

data.value_counts(data['TMIN'])

TMIN
Baja     17000
Media     8543
Alta         5
dtype: int64

In [60]:
conditions = [
          (data['PRCP'] < 1),
          ((data['PRCP'] >= 1) & (data['PRCP'] <= 2)),
          (data['PRCP'] > 2)
             ]

#define results
results = ['Poco', 'Normal', 'Mucho']

#create new column based on conditions in column1 and column2
data['PRCP'] = np.select(conditions, results)

Vemos los primeros 20 registros para comprobar que las tres columnas han sido correctamente parametrizadas.

In [61]:
data.head(20)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,Poco,Media,Baja,True
1,1948-01-02,Poco,Baja,Baja,True
2,1948-01-03,Poco,Baja,Baja,True
3,1948-01-04,Poco,Baja,Baja,True
4,1948-01-05,Poco,Baja,Baja,True
5,1948-01-06,Poco,Baja,Baja,True
6,1948-01-07,Poco,Media,Baja,True
7,1948-01-08,Poco,Baja,Baja,True
8,1948-01-09,Poco,Media,Baja,True
9,1948-01-10,Poco,Baja,Baja,True


In [62]:
data.value_counts(data['PRCP'])

PRCP
Poco      25189
Normal      327
Mucho        32
dtype: int64

#Clasificación basada en redes Bayesianas

Contamos cuantos registros tienen como clasificación True y cuantos False y lo guardamos en variables, aunque luego solo utilizaremos las de True.

Creamos un dataset de entrenamiento que contiene el 95% de los registros del dataset original y otro de test que contiene el 5% restante para poder calcular el error al final.

Hemos decidido coger este número para que la compilación de la función no tarde mucho tiempo.

In [63]:
data_train, data_test = train_test_split(data, random_state=104, test_size=0.05, shuffle=True)

# Guardamos la frecuencia del valor 'True' y 'False' en variables
total_rain_true  = data_train['RAIN'].value_counts()[True]
total_rain_false = data_train['RAIN'].value_counts()[False]

# Total de registros
total_registros = total_rain_true + total_rain_false

# P(True)
prob_true = total_rain_true/total_registros

Creamos un nuevo dataset que contiene solo los registros clasificados como 'True', para poder sacar el resto de frecuencias.

In [64]:
data_rain_true = data_train.query('RAIN != @False')
data_rain_true

,DATE,PRCP,TMAX,TMIN,RAIN
1429,1951-11-30,Poco,Media,Baja,True
16852,1994-02-20,Poco,Baja,Baja,True
16879,1994-03-19,Poco,Media,Baja,True
3942,1958-10-17,Poco,Media,Baja,True
15748,1991-02-12,Poco,Baja,Baja,True
...,...,...,...,...,...
4433,1960-02-20,Poco,Baja,Baja,True
12983,1983-07-19,Poco,Alta,Media,True
25210,2017-01-08,Poco,Baja,Baja,True
21634,2007-03-26,Poco,Media,Baja,True


###Probamos como clasifica según las probabilidades generadas

Creamos un dataset nuevo con los registros de los 3 días posteriores a los que ya tenemos en el dataset inicial, con sus correspondientes datos de Precipitaciones, Temperatura máxima y Temperatura mínima.

Se corresponden con los días '15-12-2017', '16-12-2017' y '17-12-2017'.

In [69]:
# Creamos un dataframe con los nuevos registros
registros_nuevos = pd.DataFrame({'DATE': ['15-12-2017', '16-12-2017', '17-12-2017'], 'PRCP': ['Poco', 'Mucho', 'Normal'], 'TMAX': ['Media', 'Baja', 'Alta'], 'TMIN': ['Baja', 'Media', 'Baja'], 'RAIN': [True, True, False]})

registros_nuevos.dtypes

DATE    object
PRCP    object
TMAX    object
TMIN    object
RAIN      bool
dtype: object

### Creación de las funciones


#### Función Clasificación Bayesiana
Creamos una función llamada clasificacion_bayesiana, que recopilará cada una de las probabilidades necesarias para poder finalmente calcular la probabilidad de que llueva, es decir, 'RAIN' == True del registro dado.

La función, devuelve dicha probabilidad.

En caso de que el registro no se encuentre en la base de datos de entrenamiento, decidimos que la probabilidad de lluvia sea cero, pues no podemos predecirla, pues devolvería infinito.

In [77]:
# Iteramos sobre las filas del dataframe nuevo con un bucle for
def clasificacion_bayesiana(registro, data_set, probabilidad_lluvia):

  probabilidad_PRCP = data_set['PRCP'].value_counts()[registro['PRCP']]/total_rain_true

 # Probabilidades de 'RAIN == True' en función de la TMAX.
  probabilidad_TMAX = data_set['TMAX'].value_counts()[registro['TMAX']]/total_rain_true

  # Probabilidades de 'RAIN == True' en función de la TMIN.
  probabilidad_TMIN = data_set['TMIN'].value_counts()[registro['TMIN']]/total_rain_true

  # Definimos la probabilidad del registro
  probabilidad_registro = 0

  registro = registro.to_frame().T.reset_index()
  columnas_comparar = ['PRCP', 'TMAX','TMIN']

  # Seleccionamos solo las columnas de interés del DataFrame original
  seleccion = data_train.loc[:, columnas_comparar]
  sel2 = registro.loc[:,columnas_comparar]

  # Crea el nuevo DataFrame y registro con solo las columnas seleccionadas
  nuevo_df = pd.DataFrame(seleccion)
  nuevo_registro = pd.DataFrame(sel2)
  
  # Frecuencia del registro en el dataframe de entrenamiento
  prob = len(nuevo_df[nuevo_df['PRCP'].apply(lambda x: x in nuevo_registro['PRCP'][0]) 
           & nuevo_df['TMAX'].apply(lambda x: x in nuevo_registro['TMAX'][0])
           & nuevo_df['TMIN'].apply(lambda x: x in nuevo_registro['TMIN'][0])])

  # Probabilidad del registro
  probabilidad_registro = prob/len(data_train)

  # Contemplamos que el registro no se haya dado NUNCA
  if probabilidad_registro == 0:
    print('\nNo hay registros con esta configuración en el dataframe de entrenamiento.')
    probabilidad_lluvia.append(0)
  else:
    probabilidad_lluvia.append((probabilidad_PRCP*probabilidad_TMAX*probabilidad_TMIN*prob_true)/probabilidad_registro)
  

  return probabilidad_lluvia

####Función de error
Devuelve el error del dataset completo pasado, tomando True como 1 y False como 0.
El error viene dado por la suma de los valores absoluto de la resta del valor real (0 ó 1) y el valor predicho (la probabilidad devuelta), entre el número de registros que contiene dicho data set.

In [41]:
def f_errores(errores, registro, probabilidad_lluvia):

  if registro['RAIN'] == True:
    errores.append(abs(1-probabilidad_lluvia))
  else:
    errores.append(abs(0-probabilidad_lluvia))

  suma = 0
  for valor in errores:
    suma += valor
  
  error = suma/len(errores)
  return error

#### Función Tasa de Acierto
Esta función nos devuelve la precisión del modelo, a partir de la tasa de acierto del mismo.

Se saca dividiendo los aciertos entre el total de registros predichos.

In [42]:
def t_aciertos(aciertos, registro, punto_corte,probabilidad_lluvia):

  if registro['RAIN'] == True:
    if probabilidad_lluvia > punto_corte:
      aciertos.append(1)
    else:
      aciertos.append(0)
  else:
    if probabilidad_lluvia <= punto_corte:
      aciertos.append(1)
    else:
      aciertos.append(0)
  suma = 0
  for valor in aciertos:
    suma += valor
  
  tasa_aciertos = suma/len(aciertos)
  return tasa_aciertos

###Probamos como clasifica según las probabilidades generadas

Creamos un dataset nuevo con los registros de los 3 días posteriores a los que ya tenemos en el dataset inicial, con sus correspondientes datos de Precipitaciones, Temperatura máxima y Temperatura mínima. Que crearemos manualmente.

Se corresponden con los días '15-12-2017', '16-12-2017' y '17-12-2017'.

In [73]:
# Creamos un dataframe con los nuevos registros
registros_nuevos = pd.DataFrame({'DATE': ['15-12-2017', '16-12-2017', '17-12-2017'], 'PRCP': ['Poco', 'Mucho', 'Normal'], 'TMAX': ['Media', 'Baja', 'Alta'], 'TMIN': ['Baja', 'Media', 'Baja'], 'RAIN': [True, True, False]})

registros_nuevos.dtypes

DATE    object
PRCP    object
TMAX    object
TMIN    object
RAIN      bool
dtype: object

Llamamos a la función de Clasificación Bayesiana para que nos devuelva las 3 probabilidades de los nuevos registros.

In [80]:
probabilidad = []
for i in range(len(registros_nuevos)):
  probabilidad = clasificacion_bayesiana(registros_nuevos.iloc[i], data_rain_true,probabilidad)
  print('\nLa probabilidad de lluvia del día', registros_nuevos.iloc[i]['DATE'],' es de', probabilidad[i],'\n')

print(probabilidad)


La probabilidad de lluvia del día 15-12-2017  es de 0.5306010847130428 


No hay registros con esta configuración en el dataframe de entrenamiento.

La probabilidad de lluvia del día 16-12-2017  es de 0 


No hay registros con esta configuración en el dataframe de entrenamiento.

La probabilidad de lluvia del día 17-12-2017  es de 0 

[0.5306010847130428, 0, 0]


Llamamos a la función de tasa de acierto para obtener cuanto de preciso es nuestro modelo.

In [49]:
aciertos = []

for i in range(len(registros_nuevos)):
  tasa_acierto = t_aciertos(aciertos,registros_nuevos.iloc[i],0.5, probabilidad[i])
print('\nLa tasa de acierto al calcular la probabilidad de lluvia de los registros nuevos es de', tasa_acierto,'.\n')


La tasa de acierto al calcular la probabilidad de lluvia de los registros nuevos es de 0.6666666666666666 .



Llamamos a la función Errores para obtener el función de los registros nuevos.

In [48]:
errores = []

for i in range(len(registros_nuevos)):
  error = f_errores(errores,registros_nuevos.iloc[i], probabilidad[i])
print('\nEl error al calcular la probabilidad de lluvia de los registros nuevos es de', error,'.\n')


El error al calcular la probabilidad de lluvia de los registros nuevos es de 0.4890619440707907 .



### Cálculo del error

Tomando el dataframe de validación podremos sacar cuál es la precisión y el error del modelo dado un número mayor de registros.

In [82]:
len(data_test)

1278

En concreto 1278 registros.

In [85]:
prob_test = []
for i in range(len(data_test)):
  probabilidad_test = clasificacion_bayesiana(data_test.iloc[i], data_rain_true, prob_test)
print(probabilidad_test)

[0.5306010847130428, 0.5306010847130428, 0.022466878962886547, 0.4484537538060055, 0.022466878962886547, 0.4484537538060055, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.470452484508591, 0.022466878962886547, 0.470452484508591, 0.022466878962886547, 0.5306010847130428, 0.4484537538060055, 0.5306010847130428, 0.022466878962886547, 0.4484537538060055, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.4484537538060055, 0.5306010847130428, 0.4484537538060055, 0.4484537538060055, 0.470452484508591, 0.4484537538060055, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.022466878962886547, 0.5306010847130428, 0.470452484508591, 0.4611170919709588, 0.022466878962886547, 0.5306010847130428, 0.470452484508591, 0.5306010847130428, 0.470452484508591, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.5306010847130428, 0.022466878962886547, 0.470452484508591, 0.470452484508591, 0.530601084

In [86]:
len(probabilidad_test)

1278

In [88]:
aciertos_test = []

for i in range(len(data_test)):
  tasa_acierto_test = t_aciertos(aciertos_test,data_test.iloc[i],0.5, probabilidad_test[i])
print(tasa_acierto_test)

0.5978090766823161


In [87]:
errores_test = []

for i in range(len(data_test)):
  error_test = f_errores(errores_test,data_test.iloc[i], probabilidad_test[i])
print(error_test)

0.4180168142722546


#### Variaciones para mejorar el modelo

In [276]:
probabilidad_test = []
for i in range(len(data_test)):
  probabilidad_test = clasificacion_bayesiana(data_test.iloc[i], data_rain_true, 0.4, errores, tasa_aciertos, probabilidad)
print(probabilidad_test)


No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con esta configuración en el dataframe de entrenamiento.

No hay registros con est

In [277]:
aciertos_test = []

for i in range(len(data_test)):
  tasa_acierto_test = t_aciertos(aciertos_test,data_test.iloc[i],0.4, probabilidad_test[i])
print(tasa_acierto_test)

0.51953125


In [278]:
errores_test = []

for i in range(len(data_test)):
  error_test = f_errores(errores_test,data_test.iloc[i], probabilidad_test[i])
print(error_test)

0.4785485542827773


###Conclusiones

Lo comparamos con las etiquetas verdaderas para sacar conclusiones y el error de nuestro modelo.

El error con aquellos registros que tienen como Precipitaciones la etiqueta 'Mucho', es muy grande. Esto es debido a que solo hay 32 registros con dicha etiqueta. Aunque como vemos a continuación en el conteo, todos pertenecen a registros donde SÍ ha llovido, la probabilidad de dicha precipitación es de 0.334 (19/7561), es decir, un número muy pequeño, por lo que la probabilidad de que llueva que devuelve el modelo será también muy pequeña.

In [ ]:
data_rain_true.value_counts(data_rain_true['PRCP'])

PRCP
Poco      10541
Normal      327
Mucho        32
dtype: int64

In [ ]:
data.value_counts(data['PRCP'])

PRCP
Poco      25189
Normal      327
Mucho        32
dtype: int64

Realizamos este código para demostrar que el error viene dado según el número de registros clasificados como 'RAIN' == 'False' en el dataset de test.

In [ ]:
# Registros clasificados como False en el data_test
total_false_test = len(data_test.query('RAIN != @True'))
# Número total de registros data_test
total_registros_test = len(data_test)
# Mostramos que los aciertos son solo los clasificados como False
total_aciertos = total_false_test/total_registros_test
# Cálculo del error según la hipótesis creada a partir del modelo
error = 1 - total_aciertos
print('El error obtenido manualmente es', error,'.')

El error obtenido manualmente es 0.43757338551859104 .


Podemos observar que el error obtenido manualmente es el mismo que obtuvimos al ejecutar el modelo con los datos de tipo test, 0.43757.

Podemos afirmar que solo clasifica correctamente aquellos registros que están clasificados como 'RAIN' == 'False', es decir, no llueve.

Esto es debido a que el mayor porcentaje que podemos lograr con los datos de entrenamiento tomados es que la probabilidad de lluvia sea 0.1981, mucho menor de 0.5, que es el valor a partir del cuál consideramos que el registro se clasifica como 'RAIN' == 'True'.

El porcentaje más alto se da cuando el registro tiene 'pocas' precipitaciones, temperatura alta 'media' y temperatura baja 'baja'.

###Pruebas
Intentamos bajar el error cambiando el punto de corte.

In [ ]:
clasificacion_bayesiana(data_test,0.1)

Se han truncado las últimas 5000 líneas del flujo de salida.
0.21163530347203816

La probabilidad de lluvia del día 1960-08-06  es de 0.004793036318490561 


Probabilidades del día 1961-08-30 

0.9666048237476809
0.05552610654651471
0.21163530347203816

La probabilidad de lluvia del día 1961-08-30  es de 0.004793036318490561 


Probabilidades del día 1996-12-25 

0.9666048237476809
0.32825337927378745
0.7883646965279618

La probabilidad de lluvia del día 1996-12-25  es de 0.10555085514815335 


Probabilidades del día 1985-04-10 

0.9666048237476809
0.6162205141796978
0.7883646965279618

La probabilidad de lluvia del día 1985-04-10  es de 0.19814754801732454 


Probabilidades del día 1985-09-10 

0.9666048237476809
0.6162205141796978
0.21163530347203816

La probabilidad de lluvia del día 1985-09-10  es de 0.053192407830503836 


Probabilidades del día 1983-08-19 

0.9666048237476809
0.05552610654651471
0.21163530347203816

La probabilidad de lluvia del día 1983-08-19  es de 0.0047930363

Hemos conseguido reducir el error a 0.3896. Es decir, algunos de los registros clasificados como True se han clasificado bien al bajar el punto de corte.

La conclusión es que con redes bayesianas, un modelo depende al completo de los datos con los que se tomen las probabilidades, es decir, los llamados datos de entrenamiento.

También, pensamos que a la hora de predecir si va a llover o no, serían necesarios más parámetros y una mejor distrbución de los datos, por ejemplo, registros donde hay  muchas precipitaciones, tan solo hay 32 de un dataset de 25548 registros; aunque siempre que precipitaciones = Mucho, resulta que llueve, la probabilidad que devuelve este parámetro es muy pequeña (0.00291, practicamente 0), por lo que siempre se clasificará erróneamente.